<a href="https://colab.research.google.com/github/Bhupali274/Explainable-Fake-news-Detection/blob/main/Another_copy_of_FakeNews_BERT_SHAP_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
torch.cuda.is_available()


True

In [ ]:
!pip install transformers datasets torch shap scikit-learn pandas numpy matplotlib

In [ ]:
import pandas as pd
import numpy as np
import torch

from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import shap


In [ ]:
from google.colab import files

uploaded = files.upload()

Saving true_news_dataset.ipynb to true_news_dataset (1).ipynb
Saving fake_news_dataset.ipynb to fake_news_dataset (2).ipynb


In [ ]:
import pandas as pd
import nbformat

def load_df_from_ipynb(path, var_name):
    nb = nbformat.read(path, as_version=4)
    local_vars = {}
    for cell in nb.cells:
        if cell.cell_type == "code":
            exec(cell.source, {}, local_vars)
    return local_vars[var_name]

fake_df = load_df_from_ipynb("fake_news_dataset.ipynb", "fake_df")
true_df = load_df_from_ipynb("true_news_dataset.ipynb", "true_df")

print(fake_df.shape, true_df.shape)


(20, 2) (20, 2)


In [ ]:
data = pd.concat([fake_df, true_df])
data = data.sample(frac=1).reset_index(drop=True)

data.head()


,text,label
0,Drinking salt water cures cancer says fake art...,0
1,Scientists discover water on Mars,1
2,NASA announces successful space mission,1
3,Supreme Court issues new legal guidelines,1
4,Fake claim that chocolate cures depression,0


In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    data["text"],
    data["label"],
    test_size=0.2,
    random_state=42
)


In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")

train_enc = tokenizer(
    X_train.tolist(),
    truncation=True,
    padding=True,
    max_length=128
)

test_enc = tokenizer(
    X_test.tolist(),
    truncation=True,
    padding=True,
    max_length=128
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
import torch

class NewsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels.tolist()

    def __getitem__(self, idx):
        item = {k: torch.tensor(v[idx]) for k, v in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = NewsDataset(train_enc, y_train)
test_dataset = NewsDataset(test_enc, y_test)


In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2
)


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/199 [00:00<?, ?it/s]

BertForSequenceClassification LOAD REPORT from: bert-base-uncased
Key                                        | Status     | 
-------------------------------------------+------------+-
cls.predictions.bias                       | UNEXPECTED | 
cls.predictions.transform.LayerNorm.bias   | UNEXPECTED | 
cls.predictions.transform.LayerNorm.weight | UNEXPECTED | 
cls.predictions.transform.dense.weight     | UNEXPECTED | 
cls.seq_relationship.weight                | UNEXPECTED | 
cls.predictions.transform.dense.bias       | UNEXPECTED | 
cls.seq_relationship.bias                  | UNEXPECTED | 
classifier.bias                            | MISSING    | 
classifier.weight                          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    eval_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    logging_dir="./logs",
    save_strategy="no"
)

`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


In [ ]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

trainer.train()


Epoch,Training Loss,Validation Loss
1,No log,0.597500
2,No log,0.584347


TrainOutput(global_step=8, training_loss=0.6204249858856201, metrics={'train_runtime': 2.5977, 'train_samples_per_second': 24.638, 'train_steps_per_second': 3.08, 'total_flos': 361777701120.0, 'train_loss': 0.6204249858856201, 'epoch': 2.0})

Please select `True.csv` and `Fake.csv` from your local machine when prompted. After uploading, you can re-run the cell that reads the CSV files.

In [ ]:
trainer.evaluate()


{'eval_loss': 0.5843469500541687,
 'eval_runtime': 0.0246,
 'eval_samples_per_second': 324.693,
 'eval_steps_per_second': 40.587,
 'epoch': 2.0}

In [ ]:
import torch
import numpy as np

def predict_proba(inputs_from_shap):
    # Check if inputs_from_shap are numerical token IDs (likely a numpy array)
    if isinstance(inputs_from_shap, np.ndarray) and inputs_from_shap.dtype.kind in ('i', 'u'): # 'i' for signed int, 'u' for unsigned int
        # Inputs are already token IDs, convert to tensor
        input_ids = torch.tensor(inputs_from_shap, dtype=torch.long)
        attention_mask = (input_ids != tokenizer.pad_token_id).long()
        # Add batch dimension if a single sequence is passed
        if input_ids.dim() == 1:
            input_ids = input_ids.unsqueeze(0)
            attention_mask = attention_mask.unsqueeze(0)
    elif isinstance(inputs_from_shap, (str, list, np.ndarray)):
        # Inputs are strings, list of strings, or numpy array of strings; tokenize them
        if isinstance(inputs_from_shap, np.ndarray) and inputs_from_shap.dtype.kind == 'U': # 'U' for Unicode string
             inputs_from_shap = inputs_from_shap.tolist()
        tokenized_output = tokenizer(
            inputs_from_shap,
            return_tensors="pt",
            padding=True,
            truncation=True,
            max_length=128
        )
        input_ids = tokenized_output['input_ids']
        attention_mask = tokenized_output['attention_mask']
    else:
        raise ValueError(f"Unexpected input type from SHAP masker: {type(inputs_from_shap)}")


    # Move tensors to the correct device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    input_ids = input_ids.to(device)
    attention_mask = attention_mask.to(device)
    model.to(device) # Ensure model is on the correct device

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
    return torch.softmax(outputs.logits, dim=1).cpu().numpy()

# Create a text masker using the BertTokenizer
masker = shap.maskers.Text(tokenizer)

# Initialize the explainer with the prediction function and the text masker
explainer = shap.Explainer(predict_proba, masker)

# Generate SHAP values for a subset of the test data
shap_values = explainer(X_test[:5].tolist())

shap.plots.text(shap_values)